In [2]:
import numpy as np

path = '/home/yuxuan/project/NeuralSurfaceField/Data/BuFF/buff_release_rot_const/sequences/00096/shortlong_hips/shortlong_hips_000216_cano.npy'

cano = np.load(path, allow_pickle=True).item()
cano_verts = cano['cano_points']
cano_normals = cano['cano_normals']

In [3]:
import open3d as o3d

pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(cano_verts)
pcd.normals = o3d.utility.Vector3dVector(cano_normals)

# write point cloud
o3d.io.write_point_cloud("visualization/00096_shortlong_hips_000216_cano.ply", pcd)

True

In [1]:
from dataloaders.dataloader_buff import DataLoader_Buff_depth

splt_file = '/home/yuxuan/project/NeuralSurfaceField/assets/data_split/buff_male_train_val.pkl' 
subject_index_dict = {}
subject_index_dict.update({"00032_shortlong": 0,
                            "00096_shortlong": 1})
train_dataset = DataLoader_Buff_depth(mode='train', batch_size=2, num_workers=4, split_file=splt_file, subject_index_dict=subject_index_dict)
train_loader = train_dataset.get_loader()


Loading train data...


100%|██████████| 569/569 [00:04<00:00, 137.15it/s]

Data loaded, in total 569 train examples.



In [2]:
for i, batch in enumerate(train_loader):
    break

In [9]:
import open3d as o3d

# write function which uses open3d to write point cloud
def write_pcd(path, points, normals):
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(points)
    pcd.normals = o3d.utility.Vector3dVector(normals)
    o3d.io.write_point_cloud(path, pcd)

path_cano = '/home/yuxuan/project/NeuralSurfaceField/visualization/cano.ply'
path_posed = '/home/yuxuan/project/NeuralSurfaceField/visualization/posed.ply'

# write cano point cloud
write_pcd(path_cano, batch['cano_points'][0].cpu().numpy(), batch['cano_normals'][0].cpu().numpy())
# write posed point cloud
write_pcd(path_posed, batch['scan_points'][0].cpu().numpy().transpose(), batch['scan_normals'][0].cpu().numpy().transpose())

In [5]:
import open3d as o3d
import numpy as np

dict_ = np.load('/home/yuxuan/project/NeuralSurfaceField/Data_scan/BuFF/buff_release/sequences/00159/shortshort_twist_tilt_left/shortshort_twist_tilt_left.000050.npz')

# o3d point cloud visualization
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(dict_['v_posed'])
o3d.visualization.draw_geometries([pcd])

In [3]:
batch['cano_points'].shape

torch.Size([2, 30000, 3])